# API (midbench.envs)

## Code Examples

### **2D Airfoil**

<!-- **SU2 Singularity Container**

We perform the 2D airfoil simulation and optimization using [SU2](https://su2code.github.io/) CFD solver. For user's easy usage, we have pre-built the SU2 suite using a [singularity container](https://docs.sylabs.io/guides/3.5/user-guide/introduction.html) with the required environments set up. The ready-to-use SU2 singularity container can be pulled or downloaded from https://cloud.sylabs.io/library/junideallab/midbench/su2v7.3.1_conda3.9.12_gmsh. 

* Install Singularity. If you already have singularity installed on your device, please ignore this step. If not, please follow this 3-step quick installation guide (https://docs.sylabs.io/guides/3.5/user-guide/quick_start.html ) to install Singularity first. I’m using Singularity version 3.5.3. You should be able to run the SIF container with the same version or higher. 

* Download SU2 SIF container. Please download the SU2 SIF container from Sylabs using this link (https://cloud.sylabs.io/library/junideallab/midbench/su2v7.3.1_conda3.9.12_gmsh). 


* To run the midbench API code:
    * Enter SU2 SIF container by typing the following command in your terminal: `singularity shell /path/to/your/downloaded/su2v7.3.1_conda3.9.12_gmsh`
    * Open jupyter notebook or jupyter lab by typing command in your terminal: jupyter lab. In the MIDBenchmarkSuite directory, there is an iPython notebook “midbench_api.ipynb”. Please open the notebook in jupyter lab. 
    * Please run the API demo code in the notebook cell by cell to see if the simulation and optimization can run successfully on your SU2 SIF container.

_**NOTE**_: The singuarity container also includes a 2D airfoil mesh generator `AirfoilGeometryConverter`. The generator will first convert the 2D coordinates of the airfoil curve points into mesh. The SU2 solver then takes the mesh for the follwoing CFD simulation and shape optimization. -->

#### Prediction (Inverse Design)

In [ ]:
import midbench
from midbench.envs import make
import torch
import numpy as np
from midbench.inverse.src.pred import cebgan_pred

In [ ]:
# Load input parameters (Mach number, Reynolds number, target lift coefficient)
inp_paras = np.load('./midbench/inverse/data/inp_paras_test.npy')[0,:].reshape(-1,3)

In [ ]:
# Predict optimal airfoil and angle of attack using CEBGAN inverse model
airfoils, aoas = cebgan_pred(inp_paras)

#### Simulation

In [ ]:
# Direct to the Environment of 2D airfoil
Env, Design, Condition = make("Airfoil2d-v0")

In [ ]:
# Convert airfoil coordinates to 2D mesh
designs = Design(air_coord_path = airfoils).meshgen() 

# Boundary condition setup for simulation
conditions = Condition(**{'mach':inp_paras[0,0],'reynolds':inp_paras[0,1],'lift':inp_paras[0,2],'aoa':aoas[0,0]})

# Target performances (e.g., lift and drag coefficients)
performances = ['lift', 'drag']

In [ ]:
# Execute simulation
lift, drag = Env.simulate(conditions, designs, performances, './tutorials/airfoil2d/results_simu')

#### Optimization

In [ ]:
# Convert airfoil coordinates to 2D mesh
designs = Design(air_coord_path = airfoils).meshgen()

# Boundary condition setup for optimization
conditions = Condition(**{'mach':inp_paras[0,0],'reynolds':inp_paras[0,1],'lift':inp_paras[0,2],'aoa':aoas[0,0]})

# Objectives (e.g., drag coefficient, lift/drag efficiency, optimized airfoil coordinates)
objectives = ['drag', 'ld_ratio', 'airfoil_opt']

In [ ]:
# Execute Optimization
drag, ld_ratio, airfoil_opt = Env.optimize(conditions, designs, objectives, './results_opt')